In [55]:
from quake.utils.utils import load_runcard
from pathlib import Path
import pickle
import numpy as np
from sklearn.svm import SVC
from src.quake.models.svm.utils import extract_feats, rearrange_scale
from src.quake.models.attention.train import load_and_compile_network as load_attention_network
from src.quake.models.attention.attention_dataloading import read_data as read_data_attention
from src.quake.models.cnn.cnn_dataloading import read_data as read_data_cnn
from src.quake.models.cnn.train import load_and_compile_network as load_cnn_network
from quake.dataset.generate_utils import Geometry

In [57]:
def get_features(data_folder: Path, train_folder: Path, extractor_type: str, setup: dict, seed: int = 42, run_tf_eagerly: bool = False):
    """SVM training.

    Parameters
    ----------
    data_folder: Path
        The input data folder path.
    setup: dict
        Settings dictionary.

    Raises  
    ------
    NotImplementedError
        If extractor type not one of `svm` or `attention`
    """
    setup.update({"seed": seed, "run_tf_eagerly": run_tf_eagerly})

    load_map_folder = train_folder.parent / extractor_type

    if extractor_type == "cnn":
        read_data_fn = read_data_cnn
        load_net_fn = load_cnn_network
    elif extractor_type == "attention":
        read_data_fn = read_data_attention
        load_net_fn = load_attention_network
    else:
        raise NotImplementedError(
            f"exctractor model not implemented, found: {extractor_type}"
        )

    train_generator, val_generator, test_generator = read_data_fn(
        data_folder, load_map_folder, setup, split_from_maps=True
    )

    geo = Geometry(setup["detector"])
    # extractor setup
    esetup = setup["model"][extractor_type]
    esetup.update({"ckpt": load_map_folder / f"{extractor_type}.h5"})
    network = load_net_fn(esetup, setup["run_tf_eagerly"], geo=geo)
    should_add_extra_feats = setup["model"]["svm"]["should_add_extra_feats"]
    train_features, train_labels = extract_feats(
        train_generator, network, should_add_extra_feats
    )
    val_features, val_labels = extract_feats(
        val_generator, network, should_add_extra_feats
    )
    test_features, test_labels = extract_feats(
        test_generator, network, should_add_extra_feats
    )

    # training and saving the SVMs
    dataset = rearrange_scale(
        train_features,
        val_features,
        test_features,
        setup["model"]["svm"]["should_do_scaling"],
    )
    labels = [train_labels, val_labels, test_labels]

    return dataset, labels


In [110]:
data_folder = Path('../output/tmp/data')
train_folder = Path('../output/tmp/models/svm')
setup = load_runcard("../output/tmp/cards/runcard.yaml")
dataset, labels = get_features(data_folder, train_folder, 'cnn', setup)

[INFO] (quake.cnn) Loading splitting maps from folder: ../output/tmp/models/cnn
INFO:quake.cnn:Loading splitting maps from folder: ../output/tmp/models/cnn
[INFO] (quake.cnn) Train dataset balancing: 14678 training points, of which 47.53% positives
INFO:quake.cnn:Train dataset balancing: 14678 training points, of which 47.53% positives
[INFO] (quake.cnn) Validation dataset balancing: 3145 training points, of which 47.82% positives
INFO:quake.cnn:Validation dataset balancing: 3145 training points, of which 47.82% positives
[INFO] (quake.cnn) Test dataset balancing: 3146 training points, of which 48.16% positives
INFO:quake.cnn:Test dataset balancing: 3146 training points, of which 48.16% positives
[INFO] (quake.cnn) Loading weights at ../output/tmp/models/cnn/cnn.h5
INFO:quake.cnn:Loading weights at ../output/tmp/models/cnn/cnn.h5


27/27 [==============================] - 1s 18ms/step


In [59]:
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector
import qutip

def get_spherical_coordinates(statevector, qbit):
    if qbit == 1:
        s0 = statevector[0] + statevector[2]
        s1 = statevector[1] + statevector[3]
    if qbit == 2:
        s0 = statevector[0] + statevector[1]
        s1 = statevector[2] + statevector[3]        
    r0 = np.abs(s0)
    phi0 = np.angle(s0)
    r1 = np.abs(s1)
    phi1 = np.angle(s1)

    r = np.sqrt(r0 ** 2 + r1 ** 2)
    theta = 2 * np.arccos(r0/r)
    phi = phi1 - phi0
    return [theta, phi]

def Plot_Bloch(x, featuremap, train_set, train_labels):
    fig = plt.figure(constrained_layout=True)
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    fig.set_figheight(5)
    fig.set_figwidth(5)

    b1 = qutip.Bloch(fig=fig, axes=ax1)
    b1.point_size = [1]
    b1.point_marker = ['o']
    b2 = qutip.Bloch(fig=fig, axes=ax2)
    b2.point_size = [1]
    b2.point_marker = ['o']
    pnts1 = np.zeros((len(train_labels),3))
    pnts2 = np.zeros((len(train_labels),3))
    for i, val in enumerate(train_set):
        bound_circuits = (featuremap.assign_parameters({x: val}))
        state = (Statevector.from_instruction(bound_circuits))
        spherical1 = get_spherical_coordinates(state, 1)
        x1 = np.sin(spherical1[0])*np.cos(spherical1[1])
        y1 = np.sin(spherical1[0])*np.sin(spherical1[1])
        z1 = np.cos(spherical1[0])
        spherical2 = get_spherical_coordinates(state, 2)
        x2 = np.sin(spherical2[0])*np.cos(spherical2[1])
        y2 = np.sin(spherical2[0])*np.sin(spherical2[1])
        z2 = np.cos(spherical2[0])
        pnts1[i] = [x1, y1, z1]
        pnts2[i] = [x2, y2, z2]
    b1.add_points(pnts1[train_labels == 1].T)
    b1.add_points(pnts1[train_labels == 0].T)
    b2.add_points(pnts2[train_labels == 1].T)
    b2.add_points(pnts2[train_labels == 0].T)
    b1.render()
    b2.render()
    
    return fig

In [60]:
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit import Aer
from qiskit_machine_learning.kernels import QuantumKernel


qubits = dataset[-1].shape[-1]
x = ParameterVector('x', length=qubits)
backend = Aer.get_backend('statevector_simulator')

def Z_featuremap(x, qubits, repeats):
    z_featuremap = QuantumCircuit(qubits)
    z_featuremap.initialize([1,0], 0)
    z_featuremap.initialize([1,0], 1)
    for i in range(repeats):
        z_featuremap.h(0)
        z_featuremap.h(1)
        z_featuremap.p(2*np.pi/2*x[0],0)
        z_featuremap.p(2*np.pi/2*x[1],1)
        z_featuremap.barrier()
    z_featuremap_picture = z_featuremap.draw()
    return z_featuremap, z_featuremap_picture 

def ZZ_featuremap(x, qubits, repeats):
    zz_featuremap = QuantumCircuit(qubits)
    zz_featuremap.initialize([1,0], 0)
    zz_featuremap.initialize([1,0], 1)
    for i in range(repeats):
        zz_featuremap.h(0)
        zz_featuremap.h(1)
        zz_featuremap.p(2*np.pi/2*x[0],0)
        zz_featuremap.p(2*np.pi/2*x[1],1)
        zz_featuremap.cx(0,1)
        zz_featuremap.p(2*(np.pi-np.pi/2*x[0])*(np.pi-np.pi/2*x[1]), 1)
        zz_featuremap.cx(0,1)
        zz_featuremap.barrier()
    zz_featuremap_picture = zz_featuremap.draw()
    return zz_featuremap, zz_featuremap_picture  

def Custom1_featuremap(x, qubits, repeats):
    custom1_featuremap = QuantumCircuit(qubits)
    custom1_featuremap.initialize([1,0], 0)
    custom1_featuremap.initialize([1,0], 1)
    for _ in range(repeats): 
        custom1_featuremap.h(1)
        custom1_featuremap.rx(np.pi/2*x[0], 0)
        custom1_featuremap.ry(np.pi/2*x[1], 1)
        for i in range(qubits):
            for j in range(i+1, qubits):
                custom1_featuremap.cx(i, j)
                custom1_featuremap.p(np.sin(np.pi/2*x[i]) * np.cos(np.pi/2*x[j]), j)
                custom1_featuremap.cx(i, j)
        custom1_featuremap.barrier()
    custom1_featuremap_picture = custom1_featuremap.draw()
    return custom1_featuremap, custom1_featuremap_picture  

def Custom2_featuremap(x, qubits, repeats):
    custom2_featuremap = QuantumCircuit(qubits)
    custom2_featuremap.initialize([1,0],0)
    custom2_featuremap.initialize([1,0],1)
    for _ in range(repeats):
        custom2_featuremap.rx(np.arcsin(2/np.pi*x[0]), 0)
        custom2_featuremap.rz(np.arccos(2/np.pi*2/np.pi*x[0]*x[0]), 0)
        custom2_featuremap.rx(np.arcsin(2/np.pi*x[1]), 1)
        custom2_featuremap.rz(np.arccos(2/np.pi*2/np.pi*x[1]*x[1]), 1)
        custom2_featuremap.barrier()
        custom2_featuremap.barrier()
    custom2_featuremap_picture = custom2_featuremap.draw()
    return custom2_featuremap, custom2_featuremap_picture  

In [61]:
zf, zf_draw = Z_featuremap(x, qubits, 1)
zzf, zzf_draw = ZZ_featuremap(x, qubits, 1)
c1, c1_draw = Custom1_featuremap(x, qubits, 1)
c2, c2_draw = Custom2_featuremap(x, qubits, 1)

kernel = lambda fmap, backend: QuantumKernel(feature_map = fmap, quantum_instance = backend)

maps = [zf, zzf, c1, c2]
def make_kernels(maps):
    kernels = []
    for map in maps:
        kernels.append(kernel(map, backend))
    return kernels
Quantum_Kernels = make_kernels(maps)

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit

set_train_qsvm, labels_train_qsvm = train_test_split(dataset[0], labels[0], train_size=50, random_state= 402)[
    ::2
]  

labelss = labels[1][:3000]
datasets = dataset[1][:3000]
partitions = np.append(
    -np.ones(labels_train_qsvm.shape[0], dtype=int),
    np.zeros(labelss.shape[0], dtype=int),
)

validation_idx = PredefinedSplit(partitions)

In [131]:
opt = {
    "C": [20, 50, 150, 500],
}

grid = GridSearchCV(SVC(kernel = 'precomputed'), opt, refit=True, verbose=3, cv=validation_idx)
set_train_val = np.concatenate((np.pi/2*set_train_qsvm, np.pi/2*datasets), axis=0)
labels_train_val = np.concatenate((labels_train_qsvm, labelss), axis=0)

In [132]:
grid.fit(Quantum_Kernels[3].evaluate(x_vec = set_train_val), labels_train_val)

Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV 1/1] END ..............................C=20;, score=0.660 total time=   0.0s
[CV 1/1] END ..............................C=50;, score=0.658 total time=   0.0s
[CV 1/1] END .............................C=150;, score=0.621 total time=   0.0s
[CV 1/1] END .............................C=500;, score=0.613 total time=   0.0s


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=SVC(kernel='precomputed'),
             param_grid={'C': [20, 50, 150, 500]}, verbose=3)

In [121]:
gau = SVC(C = 1, gamma = 10).fit(set_train_qsvm, labels_train_qsvm)

In [122]:
gau.score(datasets, labelss)

0.648

In [38]:
set_train_val.shape

(100, 2)